In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import torchvision 
import torchvision.datasets as vision_dsets
import torchvision.transforms as transforms 
import torch.optim as optim 
from torch.utils import data

import numpy as np
import matplotlib.pyplot as plt 

# Quiz 1: Pytorch 제공 API 연습하기
    
- Pytorch에서 데이터 디멘션을 다루는 연습입니다.
- 아래 물음표 부분을 채워주세요. 
- 셀 실행시에 두 문제 모두 답이 맞다고 나오는 것을 확인하고 넘어가주세요.
- 잘 모르는 함수에 대해서는 아래 PyTorch API 링크를 확인해주세요.
 - https://pytorch.org/docs/stable/tensors.html

In [2]:
####==== 1.1 torch.view 연습 ====####


sample = torch.randn(2000,3,64,64) 
# torch.randn()은 안에 기재된 디멘션만큼 평균 0, 표준편차 1의 난수를 파라미터로 갖는 텐서를 생성합니다.
# [2000, 3, 64, 64] dimension을 가지는 데이터라고 가정합니다.
#

sample_1 = sample.view(2000, -1) # 위 sample 을 (2000, 3*64*64)으로 바꿔보세요.
sample_2 = sample.view(2000 * 3, -1) # 위 sample 을 (2000*3, 64*64)로 바꿔보세요.

##### 정답 check ####
a = sample_1.size(0) == 2000 and sample_1.size(1) == 3*64*64 # True가 나와야 합니다.
b = sample_2.size(0) == 2000*3 and sample_2.size(1) == 64*64
if a and b:
  print("문제 1.1: 정답입니다")
else:
  print("문제 1.1: 답이 아닙니다")

  
  
####==== 1.2 torch.permute 연습 ====####

sample = torch.randn(2000,3,64,64)

sample_1 = sample.permute(0, 2, 1, 3) # 위 sample 을 디멘션 (2000,64,3,64)로 바꿔보세요.
sample_2 = sample.permute(0, 2, 3, 1).view(2000, -1, 3) # 위 sample 을 디멘션 (2000,64*64,3)로 바꿔보세요.
 

##### 정답 check ####
a = sample_1.size(0) == 2000 and sample_1.size(2) == 3 # True가 나와야 합니다.
b = sample_2.size(1) == 64*64 and sample_2.size(2) == 3
if a and b:
  print("문제 1.2: 정답입니다")
else:
  print("문제 1.2: 답이 아닙니다")


문제 1.1: 정답입니다
문제 1.2: 정답입니다


# Quiz 2: 뉴럴넷 모델링 연습



In [3]:
####==== 2. 모델링 연습====####

sample_input = torch.randn(100, 1, 28, 28)
# 빈 곳을 채워서 위의 sample 입력값에 대해 예측값이 나오도록 해봅니다.
# 최종 클래스 개수는 10 입니다.
                          
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__() 
        self.fc0 = nn.Linear(28*28, 30)
        self.fc1 = nn.Linear(30, 10)

    def forward(self, x):
        x = x.view(100, -1)
        x = torch.sigmoid(self.fc0(x))
        x = self.fc1(x) 
        return x

model = MNIST_Net()
output = model(sample_input)

##### 정답 check ####
a = (output.size(0) == 100 and output.size(1) == 10) # True가 나와야 합니다.

if a:
  print("문제 2 : 정답입니다")
else:
  print("문제 2 : 답이 아닙니다")


문제 2 : 정답입니다


# Quiz 3: Data Loader
- 강의 내용을 참고하여 다음의 데이터 로더를 정의하세요.
- 목적: 모델 학습 (즉, train_loader만)
- 데이터셋: CIFAR-10
- 배치 사이즈: 64
- 학습 시 셔플링을 합니다
- 이미지를 불러올 때 다음과 같은 변형을 가합니다.
 - 50% 확률로 horizontal flipping
 - pytorch tensor로 변환
 - 이미지 사이즈를 32 x 32에서 20 x 20으로 조정
 - Color Jittering을 사용 (torchvision.transforms.ColorJitter)
 
 

In [5]:
transform=torchvision.transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                          transforms.Resize((20, 20)),
                                          transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
                                          transforms.ToTensor()
                                          ])
cifar10 = vision_dsets.CIFAR10(root='./data', train=True, transform=transform, download=True)
train_loader = data.DataLoader(cifar10, batch_size=64, shuffle=True)

##### 정답 check ####

# 위 코드를 완성해주시기 바랍니다.
# 본 퀴즈는 정답 check snippet이 없습니다.
# 단, 아래 코드 결과가 (64, 3, 20, 20)이어야 합니다.
for img, label in train_loader:
    print(img.size())
    break

100%|██████████████████████████████████████████████████████████████▉| 170401792/170498071 [00:35<00:00, 6343763.16it/s]

torch.Size([64, 3, 20, 20])


170500096it [00:49, 6343763.16it/s]                                                                                    

# Quiz 4: Convolution filter dimension 연습 in Pytorch 
- 목적: Kernel size, Stride, Padding 에 따른 dimension 연습

In [6]:
import torch
import torch.nn as nn

In [7]:
####==== 4. Convolution filter dimension 연습 in Pytorch  ====####

sample_input = torch.randn(16, 3, 28, 28)
# 빈 곳을 채워서 위의 sample 입력값에 대해 아래와 같은 구조의 뉴럴넷이 되도록 해봅시다.
# ? 부분만 채워넣으면 됩니다. / activation 함수, 기타 다른 구조를 여기서는 생략했습니다.
# (16, 3, 28, 28) -> conv1 -> (16, 6, 28, 28)
# (16, 6, 28, 28) -> conv2 -> (16, 12, 14, 14)
# (16, 12, 14, 14) -> conv3 -> (16, 24, 7, 7)
# (16, 24, 7, 7) -> view function -> (16, 24 * 7 * 7)
# (16, 24 * 7 * 7) -> fc -> (16, 1)

class Conv_Net(nn.Module):
    def __init__(self):
        super(Conv_Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=4, stride=2, padding=1)
        self.fc = nn.Linear(24 * 7 * 7, 1)

    def forward(self, x):
        x = self.conv1(x)
        assert x.size() == (16, 6, 28, 28), "Size mismatched after conv1"
        
        x = self.conv2(x)
        assert x.size() == (16, 12, 14, 14), "Size mismatched after conv2"

        x = self.conv3(x)
        assert x.size() == (16, 24, 7, 7), "Size mismatched after conv3"
        
        x = x.view(16, 24 * 7 * 7)
        assert x.size() == (16, 24*7*7), "Size mismatched after view"
                               
        x = self.fc(x)
        return x
##### 정답 check ####
# 아래의 코드가 문제 없이 실행되면 성공!
# 참고) 아웃풋 사이즈 계산시 pytorch는 내림을 사용합니다. 
model = Conv_Net()
output = model(sample_input)
print(output.size())

torch.Size([16, 1])


# Quiz 5: Hog feature output size 계산
- 강의 내용을 참고하여 HOG 피처의 디멘션을 계산하는 함수를 만들어 주세요.
- 아래 정의되 있는 함수에서 ? 부분을 채워 주시면 됩니다.

In [10]:
from skimage.feature import hog
import numpy as np

In [24]:
####==== 5. HOG 피처 디멘션 계산하기 ====####
def get_output_dim(image_size, orient, pix_per_cell, cell_per_block):
    """
    @orient - 히스토그램에서의 bin의 개수
    @pix_per_cell - 하나의 셀에서 픽셀 단위의 width & height 
    @cell_per_block - 하나의 block에서 셀 단위의 width & height
    """
    bins_per_cell = orient * (cell_per_block ** 2)
    blocks_along_axis = np.floor(image_size // pix_per_cell)
    windows_along_axis = blocks_along_axis - cell_per_block + 1
    return bins_per_cell * (windows_along_axis ** 2)

In [33]:
##### 정답 check ####
image_size = 64
orient = 9
pix_per_cell = 8
cell_per_block = 2
img = np.random.randn(image_size, image_size)

features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                visualise=False, feature_vector=True)
our_dim = get_output_dim(image_size, orient, pix_per_cell, cell_per_block)

if features.shape[0] == our_dim:
    print("문제 5 : 정답입니다")
else:
    print("문제 5 : 답이 아닙니다")

c:\users\ironm\tf-nightly\lib\site-packages\skimage\feature\_hog.py:165: RuntimeWarning: invalid value encountered in sqrt
  image = np.sqrt(image)
c:\users\ironm\tf-nightly\lib\site-packages\skimage\feature\_hog.py:231: RuntimeWarning: invalid value encountered in remainder
  orientations, orientation_histogram)
c:\users\ironm\tf-nightly\lib\site-packages\skimage\feature\_hog.py:239: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


문제 5 : 정답입니다
